In [15]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

### Store CSVs into DataFrames

In [2]:
genre_csv_file = "Resources/movie_genres.csv"
genre_data_df = pd.read_csv(genre_csv_file)

sales_csv_file = "Resources/movie_sales.csv"
sales_data_df = pd.read_csv(sales_csv_file)

ratings_csv_file = "Resources/movie_ratings.csv"
ratings_data_df = pd.read_csv(ratings_csv_file)

/Users/ianpearlstein/.local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Clean DataFrames Before Final Merge

In [3]:
genre_data_df['year'] = genre_data_df['title'].str[-5:]
genre_data_df['year'] = genre_data_df['year'].str[:-1]
genre_data_df['title'] = genre_data_df['title'].str[:-6]

genre_data_df = genre_data_df[~genre_data_df['genres'].isin(['(no genres listed)'])]

genre_data_df[['genre','genre_1','genre_2','genre_3','genre_4','genre_5','genre_6','genre_7',
                 'genre_8','genre_9']] = genre_data_df['genres'].str.split('|',expand=True)

genre_data_df = genre_data_df[['title','year','genre']]
genre_data_df.dropna()

genre_data_df['title'] = genre_data_df['title'].str.lower()
genre_data_df['genre'] = genre_data_df['genre'].str.lower()

genre_data_df['title'] = genre_data_df['title'].str.strip()
genre_data_df['genre'] = genre_data_df['genre'].str.strip()

genre_data_df = genre_data_df[['title','genre']]

genre_data_df.head()

,title,genre
0,toy story,adventure
1,jumanji,adventure
2,grumpier old men,comedy
3,waiting to exhale,comedy
4,father of the bride part ii,comedy


In [4]:
del sales_data_df['studio']
del sales_data_df['rank']

sales_data_df['title'] = sales_data_df['title'].str.lower()

sales_data_df.isnull().values.any()

sales_data_df['title'].str.strip()

sales_data_df = sales_data_df[['title','lifetime_gross']]

sales_data_df.head()

,title,lifetime_gross
0,star wars: the force awakens,936662225
1,avatar,760507625
2,black panther,700059566
3,avengers: infinity war,678681680
4,titanic,659363944


In [5]:
ratings_data_df = ratings_data_df[['imdbID','Title','Year','Rated','imdbRating','imdbVotes']]

ratings_data_df.columns = map(str.lower, ratings_data_df.columns)

ratings_data_df = ratings_data_df.rename(columns={'imdbid': 'title_id'})

ratings_data_df = ratings_data_df.dropna().reset_index(drop=True)

ratings_data_df['title'] = ratings_data_df['title'].str.lower()
ratings_data_df['title'].str.strip()

ratings_data_df['rated'] = ratings_data_df['rated'].str.lower()
ratings_data_df = ratings_data_df[ratings_data_df.rated != 'approved']
ratings_data_df = ratings_data_df[ratings_data_df.rated != 'passed']

ratings_data_df.head()

,title_id,title,year,rated,imdbrating,imdbvotes
0,tt1090670,new found glory: this disaster - live in london,2004,al,6.3,11
1,tt0206367,the trumpet of the swan,2001,g,5.0,813
4,tt0098347,slaves of new york,1989,r,5.4,777
6,tt0069796,black snake,1973,r,5.1,460
7,tt0043064,train to tombstone,1950,not rated,4.8,41


### Merge DataFrames

In [6]:
merged_movies_df = pd.merge(ratings_data_df, genre_data_df, how='left', on=['title'])

final_movies_df = pd.merge(merged_movies_df, sales_data_df, how='left', on=['title'])

final_movies_df = final_movies_df.dropna().reset_index(drop=True)

final_movies_df.head()

,title_id,title,year,rated,imdbrating,imdbvotes,genre,lifetime_gross
0,tt0098347,slaves of new york,1989,r,5.4,777,drama,463972.0
1,tt2140619,two night stand,2014,r,6.4,"55,239",comedy,18612.0
2,tt1776196,fullmetal alchemist: the sacred star of milos,2011,not rated,7.1,"3,268",action,177802.0
3,tt1645089,inside job,2010,pg-13,8.3,"62,391",documentary,4312735.0
4,tt0109480,cops and robbersons,1994,pg,5.0,"5,609",comedy,11391093.0


### Create DataFrames to be loaded Into pgAdmin

In [24]:
genre_table_df = final_movies_df[['genre']] 
genre_table_df.drop_duplicates('genre', inplace = True)
genre_table_df = genre_table_df.reset_index(drop=True)

genre_table_df.insert(1, 'genre_id', range(1,1+len(genre_table_df)))

genre_table_df.head()

/Users/ianpearlstein/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,genre,genre_id
0,drama,1
1,comedy,2
2,action,3
3,documentary,4
4,crime,5


In [23]:
movie_rating_table_df = final_movies_df[['title_id','rated','imdbrating','imdbvotes']]
movie_rating_table_df.head()

,title_id,rated,imdbrating,imdbvotes
0,tt0098347,r,5.4,777
1,tt2140619,r,6.4,"55,239"
2,tt1776196,not rated,7.1,"3,268"
3,tt1645089,pg-13,8.3,"62,391"
4,tt0109480,pg,5.0,"5,609"


In [25]:
movie_table_df = final_movies_df[['title_id','title','year','lifetime_gross']]
movie_table_df.head()

,title_id,title,year,lifetime_gross
0,tt0098347,slaves of new york,1989,463972.0
1,tt2140619,two night stand,2014,18612.0
2,tt1776196,fullmetal alchemist: the sacred star of milos,2011,177802.0
3,tt1645089,inside job,2010,4312735.0
4,tt0109480,cops and robbersons,1994,11391093.0


### Store JSON data into a DataFrame

In [4]:
json_file = "../Resources/customer_location.json"
customer_location_df = pd.read_json(json_file)
customer_location_df.head()

,address,id,latitude,longitude,us_state
0,043 Mockingbird Place,1,39.1682,-86.5186,Indiana
1,4 Prentice Point,2,41.0938,-85.0707,Indiana
2,46 Derek Junction,3,32.7673,-96.7776,Texas
3,11966 Old Shore Place,4,39.0350,-94.3567,Missouri
4,5 Evergreen Circle,5,40.7808,-73.9772,New York


### Clean DataFrame

In [5]:
new_customer_location_df = customer_location_df[["id", "address", "us_state"]].copy()
new_customer_location_df.head()

,id,address,us_state
0,1,043 Mockingbird Place,Indiana
1,2,4 Prentice Point,Indiana
2,3,46 Derek Junction,Texas
3,4,11966 Old Shore Place,Missouri
4,5,5 Evergreen Circle,New York


### Connect to local database

In [6]:
rds_connection_string = "<insert user name>:<insert password>@localhost:5432/customer_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

### Check for tables

In [7]:
engine.table_names()

['customer_location', 'customer_name']

### Use pandas to load csv converted DataFrame into database

In [8]:
new_customer_data_df.to_sql(name='customer_name', con=engine, if_exists='append', index=False)

### Use pandas to load json converted DataFrame into database

In [9]:
new_customer_location_df.to_sql(name='customer_location', con=engine, if_exists='append', index=False)

### Confirm data has been added by querying the customer_name table
* NOTE: can also check using pgAdmin

In [10]:
pd.read_sql_query('select * from customer_name', con=engine).head()

,id,first_name,last_name
0,1,Benetta,Cancott
1,2,Lilyan,Cherry
2,3,Ezekiel,Benasik
3,4,Kennedy,Atlay
4,5,Sanford,Salmen


### Confirm data has been added by querying the customer_location table

In [11]:
pd.read_sql_query('select * from customer_location', con=engine).head()

,id,address,us_state
0,1,043 Mockingbird Place,Indiana
1,2,4 Prentice Point,Indiana
2,3,46 Derek Junction,Texas
3,4,11966 Old Shore Place,Missouri
4,5,5 Evergreen Circle,New York
